In [316]:
path="/home/flowers-user/adtool/discoveries/2024-09-06T20:09_exp_0_idx_2_seed_42/discovery.json"

import json
import sys
import os
import numpy as np

def load_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
    

discovery = load_json(path)

In [317]:
params=discovery['params']['dynamic_params']

In [318]:

# from matplotlib import pyplot as plt
# final_state = discovery['raw_output']
# final_state = np.array(final_state)
# plt.hist(final_state, bins=50)
# plt.show()

In [338]:



matrix1 = np.array([[params['a'], -params['b'], -params['c'], -params['d']],
                        [params['b'], params['a'], -params['d'], params['c']],
                        [params['c'], params['d'], params['a'], -params['b']],
                        [params['d'], -params['c'], params['b'], params['a']]],
                        dtype=np.float64)

matrix2 = np.array([[params['p'], -params['q'], -params['r'], -params['s']],
                        [params['q'], params['p'], params['s'], -params['r']],
                        [params['r'], -params['s'], params['p'], params['q']],
                        [params['s'], params['r'], -params['q'], params['p']]],
                        dtype=np.float64)



In [339]:
matrix = np.matmul(matrix1, matrix2)

In [340]:
# take the inverse of the matrix
inv_matrix = np.linalg.inv(matrix)

In [341]:
final_state=np.array(discovery['raw_output'])*100
width, height = final_state.shape

# add noise to the final state
#noise = np.random.normal(0, 1, (width, height))*0.2
# uniform between min and max
noise = np.random.uniform(final_state.min(), final_state.max(), (width, height))*0.2
#noise = noise *1000/np.sum(noise)
#final_state = final_state + noise

In [388]:
final_state=np.zeros((width,height))
# # # 16 random value in the center
# #final_state[height//2-2:height//2+2,width//2-2:width//2+2]=10
# # one single value in the center
# #final_state[height//2,width//2]=10
# # 32 random value in the center

# final_state[height//2-4:height//2+4,width//2-4:width//2+4]=np.random.random((8,8))*4

# make the initial condition a circle (not a disk)

for i in range(width):
    for j in range(height):
        if abs((i-width//2)**2+(j-height//2)**2 -2**11)<10:
            final_state[i,j]=10

# add a square in the center
final_state[height//2-4:height//2+4,width//2-4:width//2+4]=10




In [389]:
def update_block(block):
    # Flatten the block
    block = block.flatten()

    # Apply the transformation using the matrix
    new_block = matrix.dot(block)
    # 1/x
#    new_block = 1 / new_block
  #  new_block = np.tanh(new_block)

    # Reshape the block back to 2x2
    new_block = new_block.reshape(2, 2)

    return new_block

In [390]:
def _update( grid, step):
    offset = (step % 2)
    new_grid = grid.copy()

    for i in range(offset, width - 1, 2):
        for j in range(offset, height - 1, 2):
            block = grid[i:i+2, j:j+2]
            new_grid[i:i+2, j:j+2] = update_block(block)
    

    return new_grid

In [391]:
state=final_state.copy()
frames=[]

In [392]:
# add noise to 1/100 of the cells, other stays the same
# for i in range(0, width):
#     for j in range(0, height):
#         state[i, j] += 0.01*np.random.random()#*state[i, j]



In [393]:

for i in range(200):
    state = _update(state, i) #+1 pour reverse
  #  print(state.sum(   ))
  # sum of the squares
  #  print(np.sum(np.square(state)))
    frames.append(np.square(state))

In [394]:

from typing import Tuple, List
import io
import matplotlib
import imageio
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.animation import FuncAnimation


def render( data_dict, mode: str = "PIL_image") -> Tuple[bytes,str]:
    # ignores data_dict, as the render is based on self.orbit
    # in which only the last state is stored in data_dict["output"]

    colormap = create_colormap(
        np.array(
            [
                [255, 255, 255],
                [119, 255, 255],
                [23, 223, 252],
                [0, 190, 250],
                [0, 158, 249],
                [0, 142, 249],
                [81, 125, 248],
                [150, 109, 248],
                [192, 77, 247],
                [232, 47, 247],
                [255, 9, 247],
                [200, 0, 84],
            ]
        )
        / 255
        * 8
    )
    im_array = []
    for img in frames:
        # need to squeeze leading dimensions
        parsed_img = img.squeeze()
        im = im_from_array_with_colormap(parsed_img, colormap)
        im_array.append(im.convert("RGB"))

    if mode == "human":
        matplotlib.use("TkAgg")
        fig = plt.figure(figsize=(4, 4))
        animation = FuncAnimation(
            fig, lambda frame: plt.imshow(frame), frames=im_array
        )
        plt.axis("off")
        plt.tight_layout()
        return plt.show()
    elif mode == "PIL_image":
        byte_img = io.BytesIO()
        imageio.mimwrite(
            byte_img, im_array, "mp4", fps=30, output_params=["-f", "mp4"]
        )
        return [(byte_img.getvalue(), "mp4")]
    else:
        raise NotImplementedError




def create_colormap(colors: np.ndarray, is_marker_w: bool = True) -> List[int]:

    MARKER_COLORS_W = [0x5F, 0x5F, 0x5F, 0x7F, 0x7F, 0x7F, 0xFF, 0xFF, 0xFF]
    MARKER_COLORS_B = [0x9F, 0x9F, 0x9F, 0x7F, 0x7F, 0x7F, 0x0F, 0x0F, 0x0F]
    nval = 253
    ncol = colors.shape[0]
    colors = np.vstack((colors, np.array([[0, 0, 0]])))
    v = np.repeat(range(nval), 3)  # [0 0 0 1 1 1 ... 252 252 252]
    i = np.array(list(range(3)) * nval)  # [0 1 2 0 1 2 ... 0 1 2]
    k = v / (nval - 1) * (ncol - 1)  # interpolate between 0 .. ncol-1
    k1 = k.astype(int)
    c1, c2 = colors[k1, i], colors[k1 + 1, i]
    c = (k - k1) * (c2 - c1) + c1  # interpolate between c1 .. c2
    return np.rint(c / 8 * 255).astype(int).tolist() + (
        MARKER_COLORS_W if is_marker_w else MARKER_COLORS_B
    )


def im_from_array_with_colormap(np_array: np.ndarray, colormap: List[int]) -> Image:
    """
    Function that transforms the color palette of a PIL image

    input:
        - image: the PIL image to transform
        - colormap: the desired colormap
    output: the transformed PIL image
    """
    image_array = np.uint8(np_array.astype(float) * 252.0)
    transformed_image = Image.fromarray(image_array)
    transformed_image.putpalette(colormap)

    return transformed_image


In [395]:
(bytes_ext)=render(frames,  "PIL_image")
# show the video here
from IPython.display import Video

# save video to file
with open("output.mp4", "wb") as f:
    f.write(bytes_ext[0][0])
